#Pokemons:  apanhá-los todos!


<img src='https://drive.google.com/uc?id=1T3m3yy2zHs-HDsNxhmXoiJpT52Pqs9C4' width=300>

## Interpretação do problema




Dados uma grelha infinita bidimensional, uma posição inicial e instruções de direcções sequenciais que determinam um caminho, calcular o número de posições na grelha pelos quais o caminho passa (incluindo a inicial).

## Considerações teóricas

### Problemas a resolver

1.   escolher estrutura de dados para **representar as posições na grelha**, atendendo a:
    *   eficiência da actualização da posição;
    *   correspondência entre as direcções dadas ('N','S','E','O') e as instruções para a alteração da posição actual.



2.   escolher estrutura de dados para **registar as posições vistas**, atendendo a:
    *   poder registar número arbitrário de posições;
    *   tempo de inserção de uma posição e tempo de pesquisa;
    *   se é possível registar a posição na estrutura de dados escolhida em (1.), e quão eficiente é.

### Possibilidades (mais eficientes) exploradas



#### Representação das posições na grelha:

1.   `tuples`:
    *   hashable - funcionam como `key` para um `dict` ou como elementos de um `set`;
    *   lento a actualizar a posição (não é possível incrementar directamente uma das componentes).
2.   `lists` (arrays):
    *   o mais rápido a actualizar a posição (como é mutável, permite incrementação directa da componente);
    *   não é hashable nas estruturas de dados do python.
3.   `numpy arrays`:
    *   rápido a actualizar a posição (com soma directa);
    *   não é hashable.

#### Representação das posições vistas:

1.   *hash table* (`dict` e `set` no python):
    *   insere uma nova posição rapidíssimo: tempo médio `O(1)` para `sets` com até pelo menos `10^9` elementos; e pior caso `O(n)` no tamanho atual da tabela (a partir de um certo nº de elementos);
    *   permite registar uma posição sem ter que verificar se está presente (faz override);
    *   só permite registar tipos de dados *hashable* (`int`, `tuple`, `string`);
    *   usa espaço `O(n)` na quantidade total de posições vistas.

2.   *binary search tree*:
    *   insere uma nova posição muito *muito* lentamente: tempo médio `O(log(n))` e pior caso `O(n)` (por exemplo, se for sempre para cima: 'NNNNNN') no tamanho atual da árvore;
    *   permite registar uma posição aquando de verificar se está presente
    *   permite registar quaisquer tipos de dados, mas a implementação depende de como se comparam os tipos de dados
    *   usa espaço `O(n)` na quantidade total de posições vistas, mas uma implementação recursiva pode explodir a *maximum recursion depth*.

## Algoritmos

### Módulos importados

In [4]:
import random
import math
import numpy as np
import time
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


### Classes e funções auxiliares

#### Binary search tree

In [5]:
class Node:
    """
    Represents a node in a binary search tree with directly comparable data.
    """

    def __init__(self, data):

        self.left = None
        self.right = None
        self.data = data

    def insert(self, data):
        """
        Inserts a new data value in either the node or in one of its descendants.
        The algorithm keeps the node descendants' tree well formed (as a binary search tree).
        """
        if self.data:
            if data < self.data:

                # Insert data in left subtree, recursively
                if self.left is None:
                    self.left = Node(data)
                else:
                    self.left.insert(data)

            elif data > self.data:

                # Insert data in right subtree, recursively
                if self.right is None:
                    self.right = Node(data)
                else:
                    self.right.insert(data)
        else:
            self.data = data

    
    def print_tree(self):
        """
        Prints the node descendants' tree in order.
        """
        if self.left:
            self.left.print_tree()
        print( self.data),
        if self.right:
            self.right.print_tree()

    
    def get_number_of_descendants(self):
        """
        Counts the number of descendants of the node (including itself).
        """

        return 1 + (self.left.get_number_of_descendants() if self.left else 0) + (self.right.get_number_of_descendants() if self.right else 0)

#### Adição de tuples 2D




In [6]:
def add_2D_vectors(a,b):
    return (a[0] + b[0], a[1] + b[1])

#### Criação de caminho aleatório

In [7]:
def random_string_generator(length):
    """
    Generates a string comprised of 'N', 'E', 'S', 'O', with a given length.
    """
    return "".join(random.choices('NSOE', k=length))

### Classe principal

#### Grelha e algoritmos de contagem


>Notas:
>*   Cada método da classe (são 6) é um algoritmo para resolver o mesmo problema.
>*   O método da solução final apresentada é o `hierarchical_hashing_direct_algorithm()`







    




In [8]:
class Grid:
    """
    Represents a 2D infinite grid and a given path.
    """
    def __init__(self, path):

        self.initialPosition = (0,0)

        self.vector_directions = {
            'N': (0,1),
            'S': (0,-1),
            'E': (1,0),
            'O': (-1,0)
        }
        
        self.indexed_directions = {
            'N': (1,1),
            'S': (1,-1),
            'E': (0,1),
            'O': (0,-1)
        }

        self.numpy_directions = {
            'N': np.array((0,1)),
            'S': np.array((0,-1)),
            'E': np.array((1,0)),
            'O': np.array((-1,0))
        }
        
        self.path = path
    

    def hashing_naive_algorithm(self):
        """
        Counts the number of seen positions.

        The current position is represented as a tuple.
        A new position is obtained by adding two vectors (with the direction as a vector).
        The seen positions are registered with a set whose elements are the positions themselves.
        """

        current = self.initialPosition
        seenPositions = {current}

        for move in self.path:

            # Update position
            current = add_2D_vectors(current, self.vector_directions[move])

            # Register position
            seenPositions.add(current)

        return len(seenPositions)


    def hashing_direct_algorithm(self):
        """
        Counts the number of seen positions.

        The current position is represented as a list.
        A new position is obtained by incrementing the coordinate (x or y) with the unit of movement (1 or -1) (with the direction as the instruction).
        The seen positions are registered with a set whose elements are the tuple representation of the positions.
        """

        current = list(self.initialPosition)
        seenPositions = {self.initialPosition}

        for move in self.path:

            # Update position
            dir, unit = self.indexed_directions[move]
            current[dir] += unit

            # Register position
            seenPositions.add(tuple(current))
        
        return len(seenPositions)

    def hierarchical_hashing_naive_algorithm(self):
        """
        Counts the number of seen positions.

        The current position is represented as a tuple.
        A new position is obtained by adding two vectors (with the direction as a vector).
        The seen positions are registered with a dictionary such that:
            each key corresponds to a column with seen positions;
			each value corresponds to the set of rows of seen positions in the respective column (key).
        """

        current = self.initialPosition
        seenPositions = {current[0]: {current[1]}}

        for move in self.path:

            # Update position
            current = add_2D_vectors(current, self.vector_directions[move])

            # Register position
            if current[0] not in seenPositions:
                seenPositions[current[0]] = {current[1]}
            else:
                seenPositions[current[0]].add(current[1])
        
        return sum([len(s) for s in seenPositions.values()])


    # Fastest algorithm
    def hierarchical_hashing_direct_algorithm(self):
        """
        Counts the number of seen positions.

        The current position is represented as a list.
        A new position is obtained by incrementing the coordinate (x or y) with the unit of movement (1 or -1) (with the direction as the instruction).
        The seen positions are registered with a dictionary such that:
            each key corresponds to a column with seen positions;
			each value corresponds to the set of rows of seen positions in the respective column (key).
        """

        current = list(self.initialPosition)
        seenPositions = {current[0]: {current[1]}}

        for move in self.path:

            # Update position
            dir, unit = self.indexed_directions[move]
            current[dir] += unit

            # Register position
            if current[0] not in seenPositions:
                seenPositions[current[0]] = {current[1]}
            else:
                seenPositions[current[0]].add(current[1])
        
        return sum([len(s) for s in seenPositions.values()])


    def hierarchical_hashing_numpy_algorithm(self):
        """
        Counts the number of seen positions.

        The current position is represented as a numpy array.
        A new position is obtained by adding two numpy arrays (with the direction as a numpy array).
        The seen positions are registered with a dictionary such that:
            each key corresponds to a column with seen positions;
			each value corresponds to the set of rows of seen positions in the respective column (key).
        """

        current = np.array(self.initialPosition)
        seenPositions = {current[0]: {current[1]}}

        for move in self.path:

            # Update position
            current += self.numpy_directions[move]

            # Register position
            if current[0] not in seenPositions:
                seenPositions[current[0]] = {current[1]}
            else:
                seenPositions[current[0]].add(current[1])

        return sum([len(s) for s in seenPositions.values()])


    # By far the slowest algorithm
    def bst_direct_algorithm(self):
        """
        Counts the number of seen positions.

        The current position is represented as a list.
        A new position is obtained by incrementing the coordinate (x or y) with the unit of movement (1 or -1) (with the direction as the instruction).
        The seen positions (as tuples) are registered as the values in the nodes of a binary search tree with the initial position as the root.
        """

        current = list(self.initialPosition)
        root = Node(self.initialPosition)

        for move in self.path:
            
            # Update position
            dir, unit = self.indexed_directions[move]
            current[dir] += unit

            # Register position
            root.insert(tuple(current))

        return root.get_number_of_descendants()

## Análise de tempo

### Preparação de dados

#### Criação de caminhos aleatórios
>Nota: 30 caminhos com comprimentos igualmente espaçados entre `10` e `10^8`

In [9]:
paths = [random_string_generator(int(np.floor(i))) for i in np.linspace(10,10**8,30)]

In [10]:
lengths = [len(p) for p in paths]

#### Obtenção das métricas de cada algoritmo

In [11]:
alg_metrics = {}

```
1. hashing_naive_algorithm
```

In [12]:
alg_metrics['hashing_naive'] = {
    'n':[],
    'time':[]
}
for p in paths:
    grid = Grid(p)
    start = time.time()
    n = grid.hashing_naive_algorithm()
    end = time.time()
    alg_metrics['hashing_naive']['n'].append(n)
    alg_metrics['hashing_naive']['time'].append(end-start)

```
2. hashing_direct_algorithm
```



In [13]:
alg_metrics['hashing_direct'] = {
    'n':[],
    'time':[]
}
for p in paths:
    grid = Grid(p)
    start = time.time()
    n = grid.hashing_direct_algorithm()
    end = time.time()
    alg_metrics['hashing_direct']['n'].append(n)
    alg_metrics['hashing_direct']['time'].append(end-start)

```
3. hierarchical_hashing_naive_algorithm
```

In [ ]:
alg_metrics['hierarchical_hashing_naive'] = {
    'n':[],
    'time':[]
}
for p in paths:
    grid = Grid(p)
    start = time.time()
    n = grid.hierarchical_hashing_naive_algorithm()
    end = time.time()
    alg_metrics['hierarchical_hashing_naive']['n'].append(n)
    alg_metrics['hierarchical_hashing_naive']['time'].append(end-start)

```
4. hierarchical_hashing_direct_algorithm
```

In [ ]:
alg_metrics['hierarchical_hashing_direct'] = {
    'n':[],
    'time':[]
}
for p in paths:
    grid = Grid(p)
    start = time.time()
    n = grid.hierarchical_hashing_direct_algorithm()
    end = time.time()
    alg_metrics['hierarchical_hashing_direct']['n'].append(n)
    alg_metrics['hierarchical_hashing_direct']['time'].append(end-start)

```
5. hierarchical_hashing_numpy_algorithm
```

In [ ]:
alg_metrics['hierarchical_hashing_numpy'] = {
    'n':[],
    'time':[]
}
for p in paths[:9]:
    grid = Grid(p)
    start = time.time()
    n = grid.hierarchical_hashing_numpy_algorithm()
    end = time.time()
    alg_metrics['hierarchical_hashing_numpy']['n'].append(n)
    alg_metrics['hierarchical_hashing_numpy']['time'].append(end-start)

```
6. bst_direct_algorithm
```
> Nota: explode logo para caminhos de comprimento `10^7` (para os que consegue correr, é muito mais lento que os outros).

In [ ]:
alg_metrics['bst'] = {
    'n':[],
    'time':[]
}
for p in paths[0]:
    grid = Grid(p)
    start = time.time()
    n = grid.bst_direct_algorithm()
    end = time.time()
    alg_metrics['bst']['n'].append(n)
    alg_metrics['bst']['time'].append(end-start)

### Comparação de algoritmos

In [ ]:
plt.plot(lengths, alg_metrics['hashing_naive']['time'], label = 'Naive Hash')
plt.plot(lengths, alg_metrics['hashing_direct']['time'], label = 'Direct Hash')
plt.plot(lengths, alg_metrics['hierarchical_hashing_naive']['time'], label = 'Naive Hierarchical Hash')
plt.plot(lengths, alg_metrics['hierarchical_hashing_direct']['time'], label = 'Direct Hierarchical Hash')
plt.plot(lengths[:9], alg_metrics['hierarchical_hashing_numpy']['time'], label = 'Numpy Hierarchical Hash')
plt.legent()
plt.xlabel('Length of path')
plt.ylabel('Time (s)')
plt.title("Running time by path's length")
plt.show()